Q1.5 
This query aims to expand the FA-ensemblage gene set based on upstream TF binding site motif patterns.

 - FA gene set  'FA_NCBIGene_symbol.txt'
 - motif simalarity datastore 'jaspar.nt'
 - 
 

In [1]:
import csv
import yaml
import requests
import json
import re

untill the hackathon blazegraph instance is available    
I am running a local instance [described here](http://localhost:8888/notebooks/LocalBlazeGraph.ipynb)





In [2]:
bg_host = 'http://localhost:9999'
bg = bg_host + '/blazegraph/sparql?format=json&query=' 

which offers:  

    rdf:Description rdf:nodeID="service"
        rdf:type 
            rdf:resource="http://www.w3.org/ns/sparql-service-description#Service"
        endpoint 
            rdf:resource="http://localhost:9999/blazegraph/sparql"
        endpoint 
            rdf:resource="http://localhost:9999/bigdata/LBS/sparql"
            
Trying the blazegraph/sparql endpoint yields

In [3]:
x = requests.get(bg + 'SELECT ?g1 WHERE{?g1 <http://purl.obolibrary.org/obo/SO_adjacent_to> ?o} LIMIT 1')
print(x.text)

{
  "head" : {
    "vars" : [ "g1" ]
  },
  "results" : {
    "bindings" : [ {
      "g1" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/27127"
      }
    } ]
  }
}


Other queries which work in the blazegraph web interface at 
http://localhost:9999/blazegraph/#query
include
```
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>

SELECT (COUNT(?gene) as ?count)
  WHERE {
  	?gene SO:adjacent_to ?region1 .
    ?region1 RO:0002350 ?pairwise .
    ?region2 RO:0002350 ?pairwise .
    FILTER(?region1 != ?region2)
    ?gene2 SO:adjacent_to ?region2 .
    FILTER(?gene != ?gene2)
	?pairwise SWO:0000425 '1' .
  }

count
19790
```
SPARQL path notation works

```    
    select ?g1 ?g2
	    WHERE {
        ?g1 SO:adjacent_to/RO:0002350/^RO:0002350/^SO:adjacent_to ?g2 .
        FILTER(?g1 != ?g2)
      }
    LIMIT 10
```
````
g1	                                        g2
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/3646>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/23077>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/55269>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/168850>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/5635>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/11344>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/29104>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/9578>
<http://www.ncbi.nlm.nih.gov/gene/10200>	<http://www.ncbi.nlm.nih.gov/gene/4201>
<http://www.ncbi.nlm.nih.gov/gene/64854>	<http://www.ncbi.nlm.nih.gov/gene/7024>
```

```
look for genes related to FANCA

select ?g2
	WHERE {
  	NCBIGene:2175 SO:adjacent_to/RO:0002350/^RO:0002350/^SO:adjacent_to ?g2 .
    FILTER(NCBIGene:2175 != ?g2)
  }
```


```
g2
<http://www.ncbi.nlm.nih.gov/gene/8795>
<http://www.ncbi.nlm.nih.gov/gene/348995>
<http://www.ncbi.nlm.nih.gov/gene/5166>
<http://www.ncbi.nlm.nih.gov/gene/23155>
<http://www.ncbi.nlm.nih.gov/gene/7568>
<http://www.ncbi.nlm.nih.gov/gene/441381>
<http://www.ncbi.nlm.nih.gov/gene/55147>
<http://www.ncbi.nlm.nih.gov/gene/8242>
<http://www.ncbi.nlm.nih.gov/gene/54830>
<http://www.ncbi.nlm.nih.gov/gene/22976>
<http://www.ncbi.nlm.nih.gov/gene/23193>
<http://www.ncbi.nlm.nih.gov/gene/57623>
<http://www.ncbi.nlm.nih.gov/gene/6093>
<http://www.ncbi.nlm.nih.gov/gene/54929>
<http://www.ncbi.nlm.nih.gov/gene/57577>
<http://www.ncbi.nlm.nih.gov/gene/9131>
<http://www.ncbi.nlm.nih.gov/gene/54955>
<http://www.ncbi.nlm.nih.gov/gene/51645>
<http://www.ncbi.nlm.nih.gov/gene/8659>
<http://www.ncbi.nlm.nih.gov/gene/51645>
<http://www.ncbi.nlm.nih.gov/gene/6093>
<http://www.ncbi.nlm.nih.gov/gene/81551>
<http://www.ncbi.nlm.nih.gov/gene/57623>
<http://www.ncbi.nlm.nih.gov/gene/29028>
<http://www.ncbi.nlm.nih.gov/gene/90957>
<http://www.ncbi.nlm.nih.gov/gene/9517>
<http://www.ncbi.nlm.nih.gov/gene/3376>
<http://www.ncbi.nlm.nih.gov/gene/606495>
<http://www.ncbi.nlm.nih.gov/gene/606495>
<http://www.ncbi.nlm.nih.gov/gene/90957>
<http://www.ncbi.nlm.nih.gov/gene/55147>
<http://www.ncbi.nlm.nih.gov/gene/57577>
<http://www.ncbi.nlm.nih.gov/gene/7568>
<http://www.ncbi.nlm.nih.gov/gene/3376>
<http://www.ncbi.nlm.nih.gov/gene/9517>
<http://www.ncbi.nlm.nih.gov/gene/54756>
<http://www.ncbi.nlm.nih.gov/gene/23193>
<http://www.ncbi.nlm.nih.gov/gene/348995>
<http://www.ncbi.nlm.nih.gov/gene/23155>
<http://www.ncbi.nlm.nih.gov/gene/6604>
<http://www.ncbi.nlm.nih.gov/gene/81551>
<http://www.ncbi.nlm.nih.gov/gene/54785>
<http://www.ncbi.nlm.nih.gov/gene/55171>
<http://www.ncbi.nlm.nih.gov/gene/9928>
<http://www.ncbi.nlm.nih.gov/gene/6566>
<http://www.ncbi.nlm.nih.gov/gene/22976>
<http://www.ncbi.nlm.nih.gov/gene/5232>
<http://www.ncbi.nlm.nih.gov/gene/84650>
<http://www.ncbi.nlm.nih.gov/gene/5166>
<http://www.ncbi.nlm.nih.gov/gene/255426>
```



Next, move that behavior into this jupyter notebook

In [4]:
# Want to use existing translation tables when constructing SPARQL queries
yamlurl='https://raw.githubusercontent.com/TomConlin/Jaspar_FA/master/translation_tables/curie_map.yaml'
rsponse = requests.get(yamlurl)
PREFIX = yaml.load(rsponse.text)
# print(PREFIX)

yamlurl='https://raw.githubusercontent.com/TomConlin/Jaspar_FA/master/translation_tables/translation_table.yaml'
rsponse = requests.get(yamlurl)
TT = yaml.load(rsponse.text)
# print(TT)

# redecorate the curie base IRI map 
# to look like a sparql prefix namespace map  
# except the bnode: skolemIRI which is given a java 
#     "org.openrdf.query.MalformedQueryException:
# for no good reason

prefixns = ""
for p in PREFIX: 
    if len(p) > 1:
       prefixns += 'PREFIX ' + p + ': <' + PREFIX[p] + '>\n'

print(prefixns)

PREFIX OIO: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX JASPAR: <http://fantom.gsc.riken.jp/5/sstar/JASPAR_motif:>
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX GENO: <http://purl.obolibrary.org/obo/GENO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>



The Fanconi Anemia genes come as symbols/aliases in three sets [here](https://docs.google.com/spreadsheets/d/1yX-5sfrC3vrahf4_k7-5rl4Oqzm853ollIMmUo1PTc0)  
I converted them to NCBI gene identifiers and current symbols [here](https://github.com/NCATS-Tangerine/cq-notebooks/tree/master/FA_gene_sets)

In [5]:

fagene=[]

fa1 = '../FA_gene_sets/FA_1_core_complex.txt'
core_complex = {}
with open(fa1, 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:   
        (fa_gene, fa_symbol) = row
        fagene.append(fa_gene)
        core_complex[fa_gene]=fa_symbol
        
fa2 = '../FA_gene_sets/FA_2_effector_proteins.txt'
effector_proteins = {}
with open(fa2, 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:   
        (fa_gene, fa_symbol) = row
        fagene.append(fa_gene)
        effector_proteins[fa_gene]=fa_symbol
        
fa3 = '../FA_gene_sets/FA_3_associated_proteins.txt'
associated_proteins = {} 
with open(fa3, 'r') as tabfile:
    filereader = csv.reader(tabfile, delimiter='\t')
    for row in filereader:   
        (fa_gene, fa_symbol) = row
        fagene.append(fa_gene)
        associated_proteins[fa_gene]=fa_symbol

We can get all the triples patterns available in 'jaspar.nt'   
from this [GraphViz dot file](https://github.com/TomConlin/Jaspar_FA/blob/master/jaspar_target_model.gv) since we used it to guide   
generating the RDF data in the first place.  

![](https://github.com/TomConlin/Jaspar_FA/blob/master/jaspar_target_model.png?raw=true)



Composing the query by atomizing the GraphViz elements   
and selectivly translating with the same tables   
the data was generates with means  
the SPARQL query remains free of semanticly obsolete clauses.

I did this manually, but in general, the rules were roughly:
  
-    remove angle brackets
-    add trailing dot 
-    replace BNODE: with __?__
-    if the element (predicate) is a curie then wrap it in a TT lookup
-    if the object is a LITERAL wrap it in quotes
-    give the items of intrest specific __?names__
-    formating to suit sensibilities

In particular we want: 
    candidate genes which share motif similarity with FA genes
    
    <NCBIGene:fagene><SO:adjacent_to><BNODE:gene1_upstream_region>
    <BNODE:gene1_upstream_region><RO:member of><BNODE:pairwise_similarity>
    <BNODE:gene2_upstream_region><RO:member of><BNODE:pairwise_similarity>
    <NCBIGene:xyz><SO:adjacent_to><BNODE:gene2_upstream_region> 
    
    when <NCBIGene:xyz> is not <NCBIGene:fagene> 
    
we may also be interested in limiting by region extent size   
or Jaccard similarity score

    <BNODE:pairwise_similarity><SWO:Similarity score><0.73>
    <BNODE:gene1_upstream_region><GENO:has_extent><1k>
    
    

In [6]:
querystr = ' '.join(
    ['SELECT ?fagene ?candidate\nWHERE{\n',
    '?fagene', 'SO:adjacent_to', '?gene1_upstream_region .\n', 
    '?gene1_upstream_region',  TT['RO:member of'], '?pairwise_similarity .\n',
    '?gene2_upstream_region',  TT['RO:member of'], '?pairwise_similarity .\n',
    'FILTER(?gene1_upstream_region != ?gene2_upstream_region) \n', 
    '?candidate', 'SO:adjacent_to', '?gene2_upstream_region .\n',
    # '?pairwise_similarity', TT['SWO:Similarity score'], "'1' .\n", 
    '?gene1_upstream_region', TT['GENO:has_extent'], "'1k' .\n",
    'FILTER(?fagene != ?candidate)\n}'
    ]) 


query = prefixns + "\n" + querystr
print(query)

PREFIX OIO: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX JASPAR: <http://fantom.gsc.riken.jp/5/sstar/JASPAR_motif:>
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX GENO: <http://purl.obolibrary.org/obo/GENO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>

SELECT ?fagene ?candidate
WHERE{
 ?fagene SO:adjacent_to ?gene1_upstream_region .
 ?gene1_upstream_region RO:0002350 ?pairwise_similarity .
 ?gene2_upstream_region RO:0002350 ?pairwise_similarity .
 FILTER(?gene1_upstream_region != ?gene2_upstream_region) 
 ?candidate SO:adjacent_to ?gene2_upstream_region .
 ?gene1_upstream_region GENO:0000678 '1k' .
 FILTER(?fagene != ?candidate)
}


In [7]:
for x in range(len(fagene)):

    payload = {
        'format' : 'json', 
        # '$fagene':  fagene[x] , 
        # BG is not accepting the curie. Wants IRI or LITERAL
        # Invalid binding fagene with value NCBIGene:2175: NCBIGene:2175
        # Values should follow N-Triples representation (quoted literals or URIs)
        '$fagene': '<' + re.sub('NCBIGene:',PREFIX['NCBIGene'],fagene[x]) + '>',
        'query': query
    }
    response = requests.post(bg_host + '/blazegraph/sparql', data=payload)
 
    print(fagene[x], '<' + re.sub('NCBIGene:',PREFIX['NCBIGene'],fagene[x]) + '>')
    print(response.text)
    

NCBIGene:2175 <http://www.ncbi.nlm.nih.gov/gene/2175>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/2175"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/23193"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/2175"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/6566"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/2175"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/54929"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/2175"
      },
      "candidate" : {
        "type" : "uri",

NCBIGene:2189 <http://www.ncbi.nlm.nih.gov/gene/2189>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ ]
  }
}
NCBIGene:55120 <http://www.ncbi.nlm.nih.gov/gene/55120>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/55120"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/22929"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/55120"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/23548"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/55120"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/51592"
      }
  

NCBIGene:79728 <http://www.ncbi.nlm.nih.gov/gene/79728>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/79728"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/84261"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/79728"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/4913"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/79728"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/388969"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/79728"
      },
      "candidate" : {
        "type" :

NCBIGene:2072 <http://www.ncbi.nlm.nih.gov/gene/2072>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ ]
  }
}
NCBIGene:5888 <http://www.ncbi.nlm.nih.gov/gene/5888>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ ]
  }
}
NCBIGene:672 <http://www.ncbi.nlm.nih.gov/gene/672>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ ]
  }
}
NCBIGene:10459 <http://www.ncbi.nlm.nih.gov/gene/10459>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ ]
  }
}
NCBIGene:7516 <http://www.ncbi.nlm.nih.gov/gene/7516>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/7516"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/81572"
      }
   

NCBIGene:199990 <http://www.ncbi.nlm.nih.gov/gene/199990>
{
  "head" : {
    "vars" : [ "fagene", "candidate" ]
  },
  "results" : {
    "bindings" : [ {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/199990"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/51363"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/199990"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/3972"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/199990"
      },
      "candidate" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/80320"
      }
    }, {
      "fagene" : {
        "type" : "uri",
        "value" : "http://www.ncbi.nlm.nih.gov/gene/199990"
      },
      "candidate" : {
        "ty

Something is not right,  
queries which work in the BG web interface  
return no results when sent from jupyter via python's requests lib

```
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX GENO: <http://purl.obolibrary.org/obo/>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX JASPAR: <http://fantom.gsc.riken.jp/5/sstar/JASPAR_motif:>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX OIO: <http://www.geneontology.org/formats/oboInOwl#>

SELECT 'http://www.ncbi.nlm.nih.gov/gene/201254' ?candidate
WHERE{
 <http://www.ncbi.nlm.nih.gov/gene/201254> SO:adjacent_to ?gene1_upstream_region .
 ?gene1_upstream_region RO:0002350 ?pairwise_similarity .
 ?gene2_upstream_region RO:0002350 ?pairwise_similarity .
 FILTER(?gene1_upstream_region != ?gene2_upstream_region) 
 ?candidate SO:adjacent_to ?gene2_upstream_region .
 ?gene1_upstream_region GENO:0000678 '1k' .
 FILTER( <http://www.ncbi.nlm.nih.gov/gene/201254> != ?candidate)
}
```

In [8]:
print (response.headers)

print(payload['query'])

{'Transfer-Encoding': 'chunked', 'Server': 'Jetty(9.2.3.v20140905)', 'Content-Type': 'application/sparql-results+json'}
PREFIX OIO: <http://www.geneontology.org/formats/oboInOwl#>
PREFIX RO: <http://purl.obolibrary.org/obo/RO_>
PREFIX JASPAR: <http://fantom.gsc.riken.jp/5/sstar/JASPAR_motif:>
PREFIX SO: <http://purl.obolibrary.org/obo/SO_>
PREFIX GENO: <http://purl.obolibrary.org/obo/GENO_>
PREFIX SWO: <http://www.ebi.ac.uk/efo/swo/SWO_>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX NCBIGene: <http://www.ncbi.nlm.nih.gov/gene/>

SELECT ?fagene ?candidate
WHERE{
 ?fagene SO:adjacent_to ?gene1_upstream_region .
 ?gene1_upstream_region RO:0002350 ?pairwise_similarity .
 ?gene2_upstream_region RO:0002350 ?pairwise_similarity .
 FILTER(?gene1_upstream_region != ?gene2_upstream_region) 
 ?candidate SO:adjacent_to ?gene2_upstream_region .
 ?gene1_upstream_region GENO:0000678 '1k' .
 FILTER(?fagene != ?candidate)
}


viable queries will combine chains from  

     sed 's|//.*||g' jaspar_target_model.gv| awk -F'"' '/.* -> .*/ {print "     <" $2 "><" $6 "><" $4 ">"}' jaspar_target_model.gv  
     
     <NCBIGene:123><SO:adjacent_to><BNODE:gene1_upstream_region>
     <BNODE:gene1_upstream_region><rdfs:label><gene1_upstream_region>
     <BNODE:gene1_upstream_region><GENO:has_extent><1k>
     <BNODE:gene1_upstream_region><rdf:type><SO:five_prime_flanking_region>
     <BNODE:gene1_upstream_region><rdf:comment><Candidate SO:TF_binding_sites>
     <BNODE:motif_set><rdfs:label><motif_set_sig>
     <BNODE:motif_set><rdf:type><SIO:collection>
     <BNODE:motif_set><rdf:value><3 (flavors)>
     <BNODE:motif_set><OIO:hasdbxref><http:JASPAR:motif>
     <BNODE:motif_set><OIO:subset><BNODE:motif_set>
     <BNODE:motif_set><RO:has member><JASPAR:motif>
     <JASPAR:motif><rdf:type><SO:nucleotide_motif>
     <JASPAR:motif><OIO:hasdbxref><http:JASPAR:motif>
     <JASPAR:motif><rdfs:label><motif>
     <BNODE:gene1_upstream_region><RO:member of><BNODE:pairwise similarity>
     <BNODE:gene2_upstream_region><RO:member of><BNODE:pairwise similarity>
     <BNODE:pairwise similarity><rdfs:label><gene1_region gene2_region>
     <BNODE:pairwise similarity><SWO:Similarity score><0.73>
     <BNODE:pairwise similarity><rdf:type><SWO:Jaccard’s index>